In [15]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.store.memory import InMemoryStore
from dotenv import load_dotenv

load_dotenv()

True

### Identify if question is relevant to be stored in Long-Term-Memory

In [16]:
class GradeQuestion(BaseModel):
    """Boolean value to check whether a question is related to the specified topics."""

    score: str = Field(
        description="Is the question relevant? Respond with 'Yes' or 'No'."
    )

In [17]:
system = """
You are a classifier that examines the given question or statement for any personal information or preferences.
Your job is to determine whether the input contains:
1. Personal information about the user (e.g., name, occupation, location, contact details, or other identifiable information).
2. Preferences, habits, or any explicitly mentioned likes/dislikes.

If you find any such information, respond with 'Yes'. If the input does not contain any personal information or preferences, respond with 'No'.
Your response must be ONLY 'Yes' or 'No'.
"""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Here is the input: {question}"),
    ]
)

llm = ChatOpenAI(model="gpt-4o-mini")
structured_llm = llm.with_structured_output(GradeQuestion)
grader_llm = grade_prompt | structured_llm


In [18]:
grader_llm.invoke({"question": "Where is Thomas Müller from?"})

GradeQuestion(score='No')

In [19]:
grader_llm.invoke({"question": "Where is Thomas Müller from? I love playing football myself"})

GradeQuestion(score='Yes')

### Summarise question/information

In [25]:
message = "Where is Thomas Müller from? I love playing football myself."

system = """
You are an extractor focused on identifying and summarizing personal information from the given input.
Personal information includes:
1. Names of individuals.
2. Locations.
3. Hobbies, preferences, or habits explicitly mentioned by the user.
4. Any other identifiable personal details.

Your task is to:
- Extract only the personal information present in the input.
- Ignore any irrelevant or general information.
- Provide the extracted personal information as a concise, single-sentence summary.

If no personal information is found, respond with 'No personal information found.'
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", f"Extract and summarize personal information: {message}"),
    ]
)

llm = ChatOpenAI(model="gpt-4o-mini")
summarizer = prompt | llm

In [26]:
# Invoke the summarizer and print the result
result = summarizer.invoke({})
print(f"Extracted Personal Information: {result}")

Extracted Personal Information: content='Thomas Müller is mentioned, and the user loves playing football.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 138, 'total_tokens': 151, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None} id='run-fb0cacde-d003-4bf8-b6b6-4be1727217ae-0' usage_metadata={'input_tokens': 138, 'output_tokens': 13, 'total_tokens': 151, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
